# 📝 Building a Notion Integration with Python 🚀

Hey there! Welcome to this fun guide where we'll learn how to connect your Python applications with Notion! We'll create a powerful integration that lets you programmatically interact with your Notion workspace. 🌟

## 🎯 What We'll Build

We're going to create a Notion integration that can:
1. 📊 Read and write to Notion databases
2. 📑 Create and update pages
3. 🔄 Sync data between Notion and your applications

## ✅ Prerequisites

Before we dive in, make sure you have:
- 📓 A Notion account
- 🎯 A database in Notion that you want to work with

## 🔑 Part 1: Setting Up Your Notion Integration

First, let's get you set up with the necessary credentials to talk to Notion! 

### 1. Create a Notion Integration

1. 🌐 Go to [Notion's Integration page](https://www.notion.so/my-integrations)
2. 👆 Click the "New integration" button
3. 📝 Give your integration a name (like "My Python Integration")
4. 🎨 Choose an icon and color if you want (make it pretty! ✨)
5. 💫 Select the workspace where you want to use the integration
6. 🎁 Click "Submit" to create your integration

You'll receive a secret token that looks something like this:
`secret_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`

⚠️ Important Security Note! ⚠️
Keep this token safe and never share it publicly! It's like the key to your Notion kingdom! 🏰

### 2. Get Your Database ID

Now we need to find the ID of the database you want to work with:

1. 📊 Open your Notion database in the browser
2. 🔗 Look at the URL, it will look something like:
   `https://www.notion.so/workspace/[database-id]?v=...`
3. 📋 Copy the database ID part (it's a string of characters between the last '/' and '?')

For example, from this URL:
`https://www.notion.so/workspace/7749c8c4a3f34c8c8c8c8c8c8c8c8c8c?v=...`
The database ID would be: `7749c8c4a3f34c8c8c8c8c8c8c8c8c8c`

### 3. Connect Your Database

One last important step!

1. 📱 Go to your database in Notion
2. ⚙️ Click the '...' menu in the top right
3. 👥 Look for "Add connections"
4. 🔍 Find your integration and click to connect
5. ✅ Click "Confirm" to give your integration access

Now your integration has permission to access this database! 🎉

## 🛠️ Part 2: Setting Up Your Python Environment

Let's get your Python environment ready with all the tools we need! We'll use the official Notion SDK for Python to make our lives easier. 🐍

First, let's install the required package:

In [33]:
!pip install --upgrade notion-client

In [34]:
import os
from openai import OpenAI
from notion_client import Client
from dotenv import load_dotenv
from pprint import pprint # Pretty printing json

load_dotenv()

True

## 🔐 Part 3: Setting Up Environment Variables

Let's keep our secrets safe! We'll use a `.env` file to store our sensitive information. 

Create a file named `.env` in your project directory and add these lines:

In [35]:
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

## 📊 Part 4: Basic Notion Operations

Let's look at some common operations you can do with your Notion integration! Each operation is like a different superpower for your application! 🦸‍♂️

### Reading from a Database 📖

The Notion API lets you query your database like a pro! You can:
- 🔍 Filter entries based on properties
- 📋 Sort entries in any order
- 📝 Get specific properties only

In [36]:
client = Client(auth=NOTION_API_KEY)

database = client.databases.retrieve(database_id=NOTION_DATABASE_ID)
data_source_id = database["data_sources"][0]["id"]

response = client.data_sources.query(data_source_id=data_source_id)

page_id = ""

for page in response["results"]:
    page_id = response["results"][0]["id"]
    print(page_id)
    # Retrieve the blocks of the page
    page_content = client.blocks.children.list(block_id=page_id)

    # Print the content
    for block in page_content["results"]:
        pprint(block)

2ccac076-6475-813a-b733-ce8df8916719
{'archived': False,
 'created_by': {'id': '28765066-6d1f-43c4-b530-28adf9ec1b83', 'object': 'user'},
 'created_time': '2025-12-17T10:28:00.000Z',
 'has_children': False,
 'heading_3': {'color': 'default',
               'is_toggleable': False,
               'rich_text': [{'annotations': {'bold': False,
                                              'code': False,
                                              'color': 'default',
                                              'italic': False,
                                              'strikethrough': False,
                                              'underline': False},
                              'href': None,
                              'plain_text': 'The Whispering Cosmos',
                              'text': {'content': 'The Whispering Cosmos',
                                       'link': None},
                              'type': 'text'}]},
 'id': '2ccac076-6475-81a0-b314-c410f653

In [37]:
!pip install markdown-it-py

In [38]:
import markdown_it

# Helper function to convert inline children to Notion rich_text
def inline_children_to_rich_text(children):
    """Convert markdown-it inline children to Notion rich_text array"""
    rich_text = []
    if not children:
        return rich_text
    
    # Track current formatting state
    bold = False
    italic = False
    code = False
    strikethrough = False
    
    for child in children:
        if child.type == "strong_open":
            bold = True
        elif child.type == "strong_close":
            bold = False
        elif child.type == "em_open":
            italic = True
        elif child.type == "em_close":
            italic = False
        elif child.type == "code_inline":
            rich_text.append({
                "type": "text",
                "text": {"content": child.content},
                "annotations": {
                    "bold": bold,
                    "italic": italic,
                    "code": True,
                    "strikethrough": strikethrough,
                    "underline": False,
                    "color": "default"
                }
            })
        elif child.type == "s_open":
            strikethrough = True
        elif child.type == "s_close":
            strikethrough = False
        elif child.type == "text":
            rich_text.append({
                "type": "text",
                "text": {"content": child.content},
                "annotations": {
                    "bold": bold,
                    "italic": italic,
                    "code": code,
                    "strikethrough": strikethrough,
                    "underline": False,
                    "color": "default"
                }
            })
        elif child.type == "softbreak":
            rich_text.append({
                "type": "text",
                "text": {"content": "\n"},
                "annotations": {
                    "bold": False,
                    "italic": False,
                    "code": False,
                    "strikethrough": False,
                    "underline": False,
                    "color": "default"
                }
            })
    
    return rich_text

# Function to convert Markdown to Notion blocks
def markdown_to_notion_blocks(md_text):
    md = markdown_it.MarkdownIt()
    tokens = md.parse(md_text)

    blocks = []
    current_block_type = None  # Track what block we're building
    in_list = False
    
    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        if token.type == "heading_open":
            level = int(token.tag[1])  # Extract heading level (e.g., h1 -> 1)
            # Notion only supports heading_1, heading_2, heading_3
            level = min(level, 3)
            current_block_type = f"heading_{level}"
            blocks.append({
                "type": current_block_type,
                current_block_type: {"rich_text": []}
            })
        
        elif token.type == "heading_close":
            current_block_type = None
        
        elif token.type == "paragraph_open":
            if not in_list:
                current_block_type = "paragraph"
                blocks.append({"type": "paragraph", "paragraph": {"rich_text": []}})
        
        elif token.type == "paragraph_close":
            if not in_list:
                current_block_type = None
        
        elif token.type == "bullet_list_open":
            in_list = True
        
        elif token.type == "bullet_list_close":
            in_list = False
        
        elif token.type == "ordered_list_open":
            in_list = True
        
        elif token.type == "ordered_list_close":
            in_list = False
        
        elif token.type == "list_item_open":
            current_block_type = "bulleted_list_item"
            blocks.append({
                "type": "bulleted_list_item",
                "bulleted_list_item": {"rich_text": []}
            })
        
        elif token.type == "list_item_close":
            current_block_type = None
        
        elif token.type == "inline":
            # Convert inline children to rich_text with proper formatting
            rich_text = inline_children_to_rich_text(token.children)
            
            # Add rich_text to the current block
            if blocks and current_block_type:
                blocks[-1][current_block_type]["rich_text"].extend(rich_text)
        
        elif token.type == "fence" or token.type == "code_block":
            # Code blocks
            blocks.append({
                "type": "code",
                "code": {
                    "rich_text": [{"type": "text", "text": {"content": token.content.rstrip()}}],
                    "language": token.info if token.info else "plain text"
                }
            })
        
        i += 1

    return blocks

In [39]:
from notion_client import Client

# Initialize Notion client
notion = Client(auth=NOTION_API_KEY)

def add_markdown_to_notion(page_id, md_text):
    blocks = markdown_to_notion_blocks(md_text)
    
    # Send blocks to Notion
    notion.blocks.children.append(
        block_id=page_id,
        children=blocks
    )

# Example Usage
markdown_content = """
# My Notion Page
This is a **bold** text and _italic_ text.
- Bullet list item 1
- Bullet list item 2
"""

add_markdown_to_notion(page_id, markdown_content)

In [40]:
client = Client(auth=NOTION_API_KEY)

database = client.databases.retrieve(database_id=NOTION_DATABASE_ID)
data_source_id = database["data_sources"][0]["id"]

response = client.data_sources.query(
    data_source_id=data_source_id,
    filter={
        "property": "Status",
        "status": {
            "does_not_equal": "Done"
        }
    }
)

pages = []

for page in response["results"]:
    properties = page["properties"]
    print(properties)
    page_dict = {
        "name": properties.get("Name", {}).get('title', [{}])[0].get('plain_text', ""),
        "status": (properties.get("Status", {}).get('status') or {}).get('name', ""),
    }
    pages.append(page_dict)
pprint(pages)

{'Status': {'id': 'DqVQ', 'type': 'status', 'status': {'id': 'd5944600-1368-4372-a1ac-ea03eb237f25', 'name': 'Not started', 'color': 'default'}}, 'Name': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'The Whispering Cosmos', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'The Whispering Cosmos', 'href': None}]}}
{'Status': {'id': 'DqVQ', 'type': 'status', 'status': {'id': 'd5944600-1368-4372-a1ac-ea03eb237f25', 'name': 'Not started', 'color': 'default'}}, 'Name': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': '100-Word Space Story', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '100-Word Space Story', 'href': None}]}}
{'Status': {'id': 'DqVQ', 'type': 'status', 'status': {'id': 'd5944600-1368-4372-a1ac-

### Writing to a Database ✍️

You can create new pages in your database with:
- 📝 Text content
- ✅ Checkboxes
- 📅 Dates
- 👥 People mentions
- And many more property types!

In [41]:
async def write_activity(name, priority=None, deadline=None, status="Not Started", size=None, area=None):
    """
    Create a new activity in the Notion database
    
    Parameters:
        name (str): Name of the activity
        priority (str): Priority level (e.g., "High", "Medium", "Low")
        deadline (str): Deadline date in ISO format (YYYY-MM-DD)
        status (str): Status of the activity (e.g., "Not Started", "In Progress", "Done")
        size (str): Size/effort estimation (e.g., "Small", "Medium", "Large")
        area (str): Area/category of the activity
        
    Returns:
        dict: Created page object or None if failed
    """
    try:
        # 📋 Prepare the properties for the new activity
        properties = {
            "Name": {
                "title": [
                    {
                        "text": {
                            "content": name
                        }
                    }
                ]
            },
            "Status": {
                "status": {
                    "name": status
                }
            }
        }
        
        # Add optional properties if provided
        if priority:
            properties["Priority"] = {
                "select": {
                    "name": priority
                }
            }
            
        if deadline:
            properties["Deadline"] = {
                "date": {
                    "start": deadline
                }
            }
            
        if size:
            properties["Size"] = {
                "select": {
                    "name": size
                }
            }
            
        if area:
            properties["Area"] = {
                "select": {
                    "name": area
                }
            }
            
        # 📤 Create the activity in Notion
        new_page = client.pages.create(
            parent={"database_id": NOTION_DATABASE_ID},
            
            properties=properties
        )
        
        print(f"✅ Successfully created activity: {name}")
        return new_page
        
    except Exception as e:
        print(f"❌ Error creating activity in Notion: {str(e)}")
        return None



In [42]:
# 🎯 Example usage:
new_activity = await write_activity(
    name="Complete Project Documentation",
    status="Done",
)

✅ Successfully created activity: Complete Project Documentation


In [43]:
# Create a function that given a page title and a markdown text content, it creates a new page in Notion and adds the markdown text to it
def create_notion_page(page_title, markdown_content):
    # Initialize Notion client
    notion = Client(auth=NOTION_API_KEY)
    # Create a new page
    page = notion.pages.create(
        parent={"database_id": NOTION_DATABASE_ID},
        properties={"title": {"title": [{"text": {"content": page_title}}]}}
    )
    print(page)
    # Add the markdown text to the page
    add_markdown_to_notion(page["id"], markdown_content)
    return "Page created successfully"


In [44]:
create_notion_page("Test Page 123", "This is a test page 123")

{'object': 'page', 'id': '2ccac076-6475-8132-92d6-c0be53d77a6e', 'created_time': '2025-12-17T10:35:00.000Z', 'last_edited_time': '2025-12-17T10:35:00.000Z', 'created_by': {'object': 'user', 'id': '28765066-6d1f-43c4-b530-28adf9ec1b83'}, 'last_edited_by': {'object': 'user', 'id': '28765066-6d1f-43c4-b530-28adf9ec1b83'}, 'cover': None, 'icon': None, 'parent': {'type': 'data_source_id', 'data_source_id': '2ccac076-6475-8048-8f58-000b6489af03', 'database_id': '2ccac076-6475-8051-9171-e51b2938796e'}, 'archived': False, 'in_trash': False, 'is_locked': False, 'properties': {'Status': {'id': 'DqVQ', 'type': 'status', 'status': {'id': 'd5944600-1368-4372-a1ac-ea03eb237f25', 'name': 'Not started', 'color': 'default'}}, 'Name': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Test Page 123', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Test Page 123', 'h

'Page created successfully'

## 🎨 Part 5: Understanding Notion Properties

Notion uses different property types for different kinds of data. Here are the main ones you'll work with:

### Common Property Types 📋

1. **Text** 📝
   - Title
   - Rich Text
   - URL

2. **Numbers & Dates** 🔢
   - Number
   - Date
   - Created time
   - Last edited time

3. **Organization** 🗂️
   - Select
   - Multi-select
   - Status
   - Files & Media

4. **People** 👥
   - Person
   - Created by
   - Last edited by

### Property Format Tips 💡

- 📅 Dates should be in ISO format: `2024-03-21`
- ✨ Select options must exist in the database
- 👤 People are referenced by their Notion user IDs
- 🔗 URLs must include `https://` or `http://`

## 🌟 Best Practices & Tips

Here are some pro tips to make your Notion integration awesome:

### Performance Tips 🚀

1. **Batch Operations** 📦
   - Group multiple updates together
   - Use bulk operations when possible
   - Cache results when appropriate

2. **Rate Limits** ⏱️
   - Notion has rate limits
   - Add delay between requests
   - Handle rate limit errors gracefully

### Error Handling 🛡️

Always prepare for these common scenarios:
- 🔒 Authentication errors
- 📡 Network issues
- 🚫 Permission problems
- ⏳ Rate limiting

### Security Best Practices 🔐

1. **Keep Secrets Safe** 
   - Use environment variables
   - Never expose your integration token
   - Regularly rotate tokens

2. **Access Control**
   - Only request necessary permissions
   - Regularly audit database access
   - Remove unused integrations

Remember: A well-structured integration is a happy integration! 🎉